# 04_Combine_Datasets - Finale Dataset-Kombination

**Kombination aller normalisierten und angereicherten Datasets**
- Lädt die drei normalisierten und angereicherten Datasets (2018-2019, 2022, 2025)
- Prüft Schema-Kompatibilität
- Führt Qualitätsprüfungen durch
- Erstellt finales kombiniertes und angereichertes Dataset

**Input:**
- `data/processed/dataset_2018_2019_enriched.csv`
- `data/processed/dataset_2022_enriched.csv` 
- `data/processed/dataset_2025_enriched.csv`

**Output:**
- `data/processed/berlin_housing_combined_enriched_final.csv`

## 1. Setup und Imports

In [12]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

print("Bibliotheken erfolgreich importiert!")
print(f"Pandas Version: {pd.__version__}")
print("Ziel: Kombination aller normalisierten Datasets")

Bibliotheken erfolgreich importiert!
Pandas Version: 2.2.3
Ziel: Kombination aller normalisierten Datasets


## 2. Angereicherte Datasets laden

In [3]:
print("="*60)
print("ANGEREICHERTE DATASETS LADEN")
print("="*60)

# Lade alle drei angereicherten Datasets
datasets = {}
file_paths = {
    '2018_2019': 'data/processed/dataset_2018_2019_enriched.csv',
    '2022': 'data/processed/dataset_2022_enriched.csv', 
    '2025': 'data/processed/dataset_2025_enriched.csv'
}

for dataset_name, file_path in file_paths.items():
    try:
        df = pd.read_csv(file_path)
        datasets[dataset_name] = df
        print(f"✅ {dataset_name}: {len(df):,} Zeilen, {len(df.columns)} Spalten")
    except FileNotFoundError:
        print(f"❌ {dataset_name}: Datei nicht gefunden: {file_path}")
        
print(f"\nGeladen: {len(datasets)} von {len(file_paths)} Datasets")

ANGEREICHERTE DATASETS LADEN
✅ 2018_2019: 10,387 Zeilen, 15 Spalten
✅ 2022: 2,676 Zeilen, 25 Spalten
✅ 2025: 4,424 Zeilen, 15 Spalten

Geladen: 3 von 3 Datasets


## 3. Schema-Kompatibilität prüfen

In [5]:
print("="*60)
print("SCHEMA-KOMPATIBILITÄT PRÜFEN")
print("="*60)

# Definiere erwartete Standardspalten
required_columns = ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'wol', 'ortsteil_neu']

print("Erwartete Standardspalten:")
for col in required_columns:
    print(f"  - {col}")

# Prüfe jedes Dataset
schema_check = {}
for dataset_name, df in datasets.items():
    missing_cols = [col for col in required_columns if col not in df.columns]
    extra_cols = [col for col in df.columns if col not in required_columns]
    
    schema_check[dataset_name] = {
        'missing': missing_cols,
        'extra': extra_cols,
        'valid': len(missing_cols) == 0
    }
    
    print(f"\n=== DATASET {dataset_name.upper()} ===")
    print(f"Spalten: {list(df.columns)}")
    if missing_cols:
        print(f"❌ Fehlende Standardspalten: {missing_cols}")
    else:
        print(f"✅ Alle Standardspalten vorhanden")
    
    if extra_cols:
        print(f"ℹ️  Zusätzliche Spalten: {len(extra_cols)} ({extra_cols[:3]}...)")
    
# Zusammenfassung
valid_datasets = [name for name, check in schema_check.items() if check['valid']]
print(f"\n=== SCHEMA-KOMPATIBILITÄT ===")
print(f"Valide Datasets: {len(valid_datasets)}/{len(datasets)}")
if len(valid_datasets) == len(datasets):
    print("✅ Alle Datasets sind schema-kompatibel!")
else:
    print("❌ Schema-Inkompatibilitäten gefunden!")

SCHEMA-KOMPATIBILITÄT PRÜFEN
Erwartete Standardspalten:
  - price
  - size
  - district
  - rooms
  - year
  - dataset_id
  - source
  - wol
  - ortsteil_neu

=== DATASET 2018_2019 ===
Spalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'street', 'floor', 'typeOfFlat', 'yearConstructed', 'totalRent', 'strasse', 'wol', 'ortsteil_neu']
✅ Alle Standardspalten vorhanden
ℹ️  Zusätzliche Spalten: 6 (['street', 'floor', 'typeOfFlat']...)

=== DATASET 2022 ===
Spalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'plz', 'warmmiete', 'nebenkosten', 'kaution', 'baujahr', 'zustand', 'energieeffiziensklasse', 'ausstattung_möbliert', 'ausstattung_balkon', 'ausstattung_terrasse', 'ausstattung_garten', 'ausstattung_einbauküche', 'ausstattung_garage', 'ausstattung_stellplatz', 'ausstattung_personenaufzug', 'ausstattung_keller', 'wol', 'ortsteil_neu']
✅ Alle Standardspalten vorhanden
ℹ️  Zusätzliche Spalten: 16 (['plz', 'warmmiete', 'nebenkosten']..

## 4. Datenqualität prüfen

In [6]:
print("="*60)
print("DATENQUALITÄT PRÜFEN")
print("="*60)

quality_report = {}

for dataset_name, df in datasets.items():
    print(f"\n=== DATASET {dataset_name.upper()} ===")
    
    # Grundlegende Statistiken
    total_rows = len(df)
    
    # Vollständigkeit prüfen
    completeness = {}
    for col in required_columns:
        if col in df.columns:
            non_null = df[col].notna().sum()
            completeness[col] = (non_null, non_null/total_rows*100)
            print(f"  {col}: {non_null:,}/{total_rows:,} ({non_null/total_rows*100:.1f}%) nicht-null")
    
    # Wertebereichs-Prüfungen
    if 'price' in df.columns:
        price_valid = ((df['price'] >= 50) & (df['price'] <= 20000)).sum()
        print(f"  Preis (50-20000€): {price_valid:,}/{total_rows:,} ({price_valid/total_rows*100:.1f}%) gültig")
    
    if 'size' in df.columns:
        size_valid = ((df['size'] >= 5) & (df['size'] <= 1000)).sum()
        print(f"  Größe (5-1000m²): {size_valid:,}/{total_rows:,} ({size_valid/total_rows*100:.1f}%) gültig")
    
    if 'district' in df.columns:
        unique_districts = df['district'].nunique()
        print(f"  Einzigartige Bezirke: {unique_districts}")
    
    quality_report[dataset_name] = {
        'total_rows': total_rows,
        'completeness': completeness,
        'unique_districts': df['district'].nunique() if 'district' in df.columns else 0
    }

print(f"\n✅ Datenqualitätsprüfung abgeschlossen")

DATENQUALITÄT PRÜFEN

=== DATASET 2018_2019 ===
  price: 10,387/10,387 (100.0%) nicht-null
  size: 10,387/10,387 (100.0%) nicht-null
  district: 10,387/10,387 (100.0%) nicht-null
  rooms: 10,387/10,387 (100.0%) nicht-null
  year: 10,387/10,387 (100.0%) nicht-null
  dataset_id: 10,387/10,387 (100.0%) nicht-null
  source: 10,387/10,387 (100.0%) nicht-null
  wol: 1,760/10,387 (16.9%) nicht-null
  ortsteil_neu: 1,760/10,387 (16.9%) nicht-null
  Preis (50-20000€): 10,387/10,387 (100.0%) gültig
  Größe (5-1000m²): 10,387/10,387 (100.0%) gültig
  Einzigartige Bezirke: 79

=== DATASET 2022 ===
  price: 2,676/2,676 (100.0%) nicht-null
  size: 2,676/2,676 (100.0%) nicht-null
  district: 2,676/2,676 (100.0%) nicht-null
  rooms: 2,676/2,676 (100.0%) nicht-null
  year: 2,676/2,676 (100.0%) nicht-null
  dataset_id: 2,676/2,676 (100.0%) nicht-null
  source: 2,676/2,676 (100.0%) nicht-null
  wol: 2,676/2,676 (100.0%) nicht-null
  ortsteil_neu: 2,676/2,676 (100.0%) nicht-null
  Preis (50-20000€): 2,676

## 5. Datasets kombinieren

In [7]:
print("="*60)
print("DATASETS KOMBINIEREN")
print("="*60)

# Nur Standardspalten für Kombination verwenden
datasets_standard = {}
for dataset_name, df in datasets.items():
    # Wähle nur Standardspalten aus
    available_std_cols = [col for col in required_columns if col in df.columns]
    df_std = df[available_std_cols].copy()
    datasets_standard[dataset_name] = df_std
    print(f"{dataset_name}: {len(df_std):,} Zeilen mit {len(available_std_cols)} Standardspalten")

# Kombiniere alle Datasets
print(f"\nKombiniere Datasets...")
combined_df = pd.concat(datasets_standard.values(), ignore_index=True, sort=False)

print(f"✅ Kombiniertes Dataset erstellt: {len(combined_df):,} Zeilen")

# Zusammenfassung
print(f"\n=== KOMBINATIONS-ZUSAMMENFASSUNG ===")
total_input_rows = sum(len(df) for df in datasets.values())
print(f"Input-Zeilen gesamt: {total_input_rows:,}")
print(f"Output-Zeilen: {len(combined_df):,}")
print(f"Datenverlust: {total_input_rows - len(combined_df):,} ({(total_input_rows - len(combined_df))/total_input_rows*100:.1f}%)")

# Verteilung nach Jahren
print(f"\n=== JAHRESVERTEILUNG ===")
year_counts = combined_df['year'].value_counts().sort_index()
for year, count in year_counts.items():
    print(f"  {year}: {count:,} Einträge ({count/len(combined_df)*100:.1f}%)")

# Verteilung nach Dataset-ID
print(f"\n=== DATASET-VERTEILUNG ===")
dataset_counts = combined_df['dataset_id'].value_counts()
for dataset_id, count in dataset_counts.items():
    print(f"  {dataset_id}: {count:,} Einträge ({count/len(combined_df)*100:.1f}%)")

DATASETS KOMBINIEREN
2018_2019: 10,387 Zeilen mit 9 Standardspalten
2022: 2,676 Zeilen mit 9 Standardspalten
2025: 4,424 Zeilen mit 9 Standardspalten

Kombiniere Datasets...
✅ Kombiniertes Dataset erstellt: 17,487 Zeilen

=== KOMBINATIONS-ZUSAMMENFASSUNG ===
Input-Zeilen gesamt: 17,487
Output-Zeilen: 17,487
Datenverlust: 0 (0.0%)

=== JAHRESVERTEILUNG ===
  2019: 10,387 Einträge (59.4%)
  2022: 2,676 Einträge (15.3%)
  2025: 4,424 Einträge (25.3%)

=== DATASET-VERTEILUNG ===
  historical: 10,387 Einträge (59.4%)
  recent: 4,424 Einträge (25.3%)
  current: 2,676 Einträge (15.3%)


## 6. Finale Datenvalidierung

In [9]:
print("="*60)
print("FINALE DATENVALIDIERUNG")
print("="*60)

# Duplikate prüfen
duplicates = combined_df.duplicated().sum()
print(f"Duplikate: {duplicates:,} ({duplicates/len(combined_df)*100:.2f}%)")

# Fehlende Werte
print(f"\n=== FEHLENDE WERTE ===")
missing_summary = combined_df.isnull().sum()
for col, missing_count in missing_summary.items():
    if missing_count > 0:
        print(f"  {col}: {missing_count:,} ({missing_count/len(combined_df)*100:.1f}%)")

# Statistiken der Kernfelder
print(f"\n=== STATISTIKEN KOMBINIERTES DATASET ===")
if 'price' in combined_df.columns:
    price_stats = combined_df['price'].describe()
    print(f"Preis - Min: {price_stats['min']:.0f}€, Max: {price_stats['max']:.0f}€, Median: {price_stats['50%']:.0f}€")

if 'size' in combined_df.columns:
    size_stats = combined_df['size'].describe()
    print(f"Größe - Min: {size_stats['min']:.0f}m², Max: {size_stats['max']:.0f}m², Median: {size_stats['50%']:.0f}m²")

if 'rooms' in combined_df.columns:
    rooms_stats = combined_df['rooms'].describe()
    print(f"Zimmer - Min: {rooms_stats['min']:.1f}, Max: {rooms_stats['max']:.1f}, Median: {rooms_stats['50%']:.1f}")

# Bezirksverteilung
if 'district' in combined_df.columns:
    print(f"\n=== FINALE BEZIRKSVERTEILUNG ===")
    district_counts = combined_df['district'].value_counts()
    print(f"Anzahl Bezirke: {len(district_counts)}")
    for district, count in district_counts.head(10).items():
        print(f"  {district}: {count:,} Einträge ({count/len(combined_df)*100:.1f}%)")

print(f"\n✅ Finale Datenvalidierung abgeschlossen")

FINALE DATENVALIDIERUNG
Duplikate: 1,233 (7.05%)

=== FEHLENDE WERTE ===
  rooms: 4,424 (25.3%)
  wol: 12,997 (74.3%)
  ortsteil_neu: 8,629 (49.3%)

=== STATISTIKEN KOMBINIERTES DATASET ===
Preis - Min: 150€, Max: 9990€, Median: 931€
Größe - Min: 10m², Max: 482m², Median: 69m²
Zimmer - Min: 1.0, Max: 10.0, Median: 2.0

=== FINALE BEZIRKSVERTEILUNG ===
Anzahl Bezirke: 87
  Mitte: 1,923 Einträge (11.0%)
  Pankow: 1,189 Einträge (6.8%)
  Neukölln: 904 Einträge (5.2%)
  Spandau: 837 Einträge (4.8%)
  Tiergarten: 832 Einträge (4.8%)
  Charlottenburg: 792 Einträge (4.5%)
  Friedrichshain-Kreuzberg: 775 Einträge (4.4%)
  Friedrichshain: 666 Einträge (3.8%)
  Charlottenburg-Wilmersdorf: 636 Einträge (3.6%)
  Reinickendorf: 614 Einträge (3.5%)

✅ Finale Datenvalidierung abgeschlossen


## 7. Export des finalen Datasets

In [10]:
print("="*60)
print("EXPORT FINALES KOMBINIERTES DATASET")
print("="*60)

# Export
output_file = 'data/processed/berlin_housing_combined_enriched_final.csv'
combined_df.to_csv(output_file, index=False)

print(f"✅ Finales Dataset exportiert: {output_file}")
print(f"Dateigröße: {len(combined_df):,} Zeilen x {len(combined_df.columns)} Spalten")

# Validierung durch Wiedereinlesen
test_df = pd.read_csv(output_file)
print(f"✅ Export-Validierung erfolgreich: {len(test_df):,} Zeilen geladen")

# Finale Zusammenfassung
print(f"\n=== FINALE ZUSAMMENFASSUNG ===")
print(f"Input-Datasets: {len(datasets)}")
print(f"  - Dataset 2018-2019: {quality_report.get('2018_2019', {}).get('total_rows', 0):,} Zeilen")
print(f"  - Dataset 2022: {quality_report.get('2022', {}).get('total_rows', 0):,} Zeilen") 
print(f"  - Dataset 2025: {quality_report.get('2025', {}).get('total_rows', 0):,} Zeilen")
print(f"Output: {output_file} ({len(combined_df):,} Zeilen)")
print(f"Zeitspanne: 2018-2025 ({len(combined_df['year'].unique())} Jahre)")
print(f"Berliner Bezirke: {len(combined_df['district'].unique())} abgedeckt")
print(f"Standardisierte Spalten: {list(combined_df.columns)}")

print(f"\n🎯 DATASET-KOMBINATION ERFOLGREICH ABGESCHLOSSEN!")
print(f"Das finale Dataset ist bereit für die Mietpreis-Analyse.")

EXPORT FINALES KOMBINIERTES DATASET
✅ Finales Dataset exportiert: data/processed/berlin_housing_combined_enriched_final.csv
Dateigröße: 17,487 Zeilen x 9 Spalten
✅ Export-Validierung erfolgreich: 17,487 Zeilen geladen

=== FINALE ZUSAMMENFASSUNG ===
Input-Datasets: 3
  - Dataset 2018-2019: 10,387 Zeilen
  - Dataset 2022: 2,676 Zeilen
  - Dataset 2025: 4,424 Zeilen
Output: data/processed/berlin_housing_combined_enriched_final.csv (17,487 Zeilen)
Zeitspanne: 2018-2025 (3 Jahre)
Berliner Bezirke: 87 abgedeckt
Standardisierte Spalten: ['price', 'size', 'district', 'rooms', 'year', 'dataset_id', 'source', 'wol', 'ortsteil_neu']

🎯 DATASET-KOMBINATION ERFOLGREICH ABGESCHLOSSEN!
Das finale Dataset ist bereit für die Mietpreis-Analyse.
